In [1]:
#!pip3 install twilio

In [4]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [5]:
query = 'México'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=b8e3f5e4916c49e0957201325230107&q=México&days=1&aqi=no&alerts=no'

In [ ]:
response = requests.get(url_clima).json()

In [ ]:
response

{'location': {'name': 'Tlalnepantla',
  'region': 'México',
  'country': 'Mexico',
  'lat': 19.54,
  'lon': -99.2,
  'tz_id': 'America/Mexico_City',
  'localtime_epoch': 1688243047,
  'localtime': '2023-07-01 14:24'},
 'current': {'last_updated_epoch': 1688242500,
  'last_updated': '2023-07-01 14:15',
  'temp_c': 24.0,
  'temp_f': 75.2,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 8.1,
  'wind_kph': 13.0,
  'wind_degree': 80,
  'wind_dir': 'E',
  'pressure_mb': 1026.0,
  'pressure_in': 30.3,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 39,
  'cloud': 75,
  'feelslike_c': 25.2,
  'feelslike_f': 77.3,
  'vis_km': 13.0,
  'vis_miles': 8.0,
  'uv': 6.0,
  'gust_mph': 3.4,
  'gust_kph': 5.4},
 'forecast': {'forecastday': [{'date': '2023-07-01',
    'date_epoch': 1688169600,
    'day': {'maxtemp_c': 24.0,
     'maxtemp_f': 75.2,
     'mintemp_c': 15.2,
     'mintemp_f': 59.4,
    

In [8]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [9]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

# Dataframe

In [21]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [22]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [23]:
datos[0]

('2023-07-01', 0, 'Partly cloudy', 17.2, 0, 0)

In [24]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-07-01,0,Partly cloudy,17.2,0,0
1,2023-07-01,1,Patchy rain possible,16.8,0,69
2,2023-07-01,2,Partly cloudy,16.3,0,0
3,2023-07-01,3,Partly cloudy,15.8,0,0
4,2023-07-01,4,Light rain shower,15.9,0,60
5,2023-07-01,5,Overcast,15.5,0,0
6,2023-07-01,6,Overcast,15.3,0,0
7,2023-07-01,7,Overcast,15.2,0,0
8,2023-07-01,8,Overcast,15.3,0,0
9,2023-07-01,9,Partly cloudy,16.0,0,0


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [29]:
df[df['Lluvia']==0]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-07-01,0,Partly cloudy,17.2,0,0
1,2023-07-01,1,Patchy rain possible,16.8,0,69
2,2023-07-01,2,Partly cloudy,16.3,0,0
3,2023-07-01,3,Partly cloudy,15.8,0,0
4,2023-07-01,4,Light rain shower,15.9,0,60
5,2023-07-01,5,Overcast,15.5,0,0
6,2023-07-01,6,Overcast,15.3,0,0
7,2023-07-01,7,Overcast,15.2,0,0
8,2023-07-01,8,Overcast,15.3,0,0
9,2023-07-01,9,Partly cloudy,16.0,0,0


In [30]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>5) & (df['Hora']< 23)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [31]:
df['Fecha'][0]

'2023-07-01'

In [32]:
df_rain

,Condicion
Hora,
16,Light rain shower
22,Light rain shower


In [40]:
print(f'Hola! \nEl pronostico del tiempo hoy: {df["Fecha"][0]}, en {query} es : \n{str(df_rain)}')

Hola! 
El pronostico del tiempo hoy: 2023-07-01, en México es : 
              Condicion
Hora                   
16    Light rain shower
22    Light rain shower


In [49]:
PHONE_NUMBER

'+14846998988'

# Mensaje SMS desde Twilio

In [51]:
# Download the helper library from https://www.twilio.com/docs/python/install
# import os
# from twilio.rest import Client
# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN
client = Client(account_sid, auth_token)

message = client.messages \
                .create(
    body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
    from_='whatsapp:+14155238886',
    to='whatsapp:+5217751425295'
)

print(f'Mensaje enviado {message.sid}')


Mensaje enviado SM7df2d1874e0dfd8d35e18f88fe25dea8


In [32]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+527751425295'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMbc309ab2afc09ea1f87a7eb64fb8fdda


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />